In [1]:
x1=[1,2,3]
x2=[4,5,6]

X=[x1,x2,x1,x2]

In [2]:
for i in X:
    print(i)
    i[1]=0

[1, 2, 3]
[4, 5, 6]
[1, 0, 3]
[4, 0, 6]


In [2]:
x=[1,2,3]
y=[4,5,6]
z=[]
z+=x+y
z

[1, 2, 3, 4, 5, 6]

In [18]:
import numpy as np
n=10
A=np.arange(n**2).reshape((n,n))
B=np.dot(np.transpose(A),A)
np.linalg.pinv(B)

array([[ 1.24782369e-03,  9.78071625e-04,  7.08319559e-04,
         4.38567493e-04,  1.68815427e-04, -1.00936639e-04,
        -3.70688705e-04, -6.40440771e-04, -9.10192837e-04,
        -1.17994490e-03],
       [ 9.78071625e-04,  7.66648301e-04,  5.55224977e-04,
         3.43801653e-04,  1.32378329e-04, -7.90449954e-05,
        -2.90468320e-04, -5.01891644e-04, -7.13314968e-04,
        -9.24738292e-04],
       [ 7.08319559e-04,  5.55224977e-04,  4.02130395e-04,
         2.49035813e-04,  9.59412305e-05, -5.71533517e-05,
        -2.10247934e-04, -3.63342516e-04, -5.16437098e-04,
        -6.69531680e-04],
       [ 4.38567493e-04,  3.43801653e-04,  2.49035813e-04,
         1.54269972e-04,  5.95041322e-05, -3.52617080e-05,
        -1.30027548e-04, -2.24793388e-04, -3.19559229e-04,
        -4.14325069e-04],
       [ 1.68815427e-04,  1.32378329e-04,  9.59412305e-05,
         5.95041322e-05,  2.30670340e-05, -1.33700643e-05,
        -4.98071625e-05, -8.62442608e-05, -1.22681359e-04,
        -1.

In [23]:
test=[[1,2,3],[4,5,6],[7,8,9]]
l=test[0]
l.remove(2)
test

[[1, 3], [4, 5, 6], [7, 8, 9]]

In [3]:
f=lambda x:2*x



f([0])

[0, 0]

In [4]:
import numpy as np
from scipy.optimize import fmin_slsqp

T_h_in,T_c_in=550,300

R_h,R_c=0.955299519035398,0.009137371917789051
C_h,C_c=21.176851499999994,264.31750559999995

s=0.123
k=1.5
r=4.5

alpha_h=np.array([0.9+i/1000 for i in range(200)])
alpha_c=np.array([0.9+i/1000 for i in range(200)])

def cons1(x):
    return T_h_in-x[0]

def cons2(x):
    return x[1]-T_c_in

def cons3(x):
    return x[0]-x[1]

def cons4(x):
    return x[0]

def cons5(x):
    return x[1]

result=[]

for i in alpha_h:
    for j in alpha_c:
                
        def f(x):
            T_h_mean=(T_h_in+x[0])*0.5*i
            T_c_mean=(T_c_in+x[1])*0.5*j

            q_h=(T_h_in-x[0])*C_h
            q_c=(x[1]-T_c_in)*C_c

            T_TEM_h=T_h_mean-q_h*R_h
            T_TEM_c=T_c_mean+q_c*R_c

            delT=T_TEM_h-T_TEM_c

            I=s*delT/2/r

            q_h_E=s*I*T_TEM_h+k*delT-0.5*I**2*r
            q_c_E=s*I*T_TEM_c+k*delT+0.5*I**2*r

            return (q_h-q_h_E)**2+(q_c-q_c_E)**2

        result.append([i,j,fmin_slsqp(f,[T_h_in,T_c_in],ieqcons=[cons1,cons2,cons3,cons4,cons5],iprint=-1)])


In [5]:
def g(arr):
    t=arr[2]
    T_h_out=t[0]
    T_c_out=t[1]

    q_h=(T_h_in-T_h_out)*C_h
    q_c=(T_c_out-T_c_in)*C_c
    return [arr[0],arr[1],q_h-q_c,t[0],t[1]]

anal=[]

for i in result:
    anal.append(g(i))

import pandas as pd

df=pd.DataFrame(anal,columns=['alpha_h','alpha_c','W','T_h_out','T_c_out'])
df['W_%']=df['W']/df['W'].min()
df['T_h_out_%']=df['T_h_out']/df['T_h_out'].min()
df['T_c_out_%']=df['T_c_out']/df['T_c_out'].min()
df


,alpha_h,alpha_c,W,T_h_out,T_c_out,W_%,T_h_out_%,T_c_out_%
0,0.900,0.900,4.653234,542.795640,300.559601,1.943511,1.006577,1.000466
1,0.900,0.901,4.639584,542.804836,300.558916,1.937810,1.006594,1.000463
2,0.900,0.902,4.626171,542.814029,300.558230,1.932207,1.006611,1.000461
3,0.900,0.903,4.612670,542.823226,300.557545,1.926568,1.006628,1.000459
4,0.900,0.904,4.599255,542.832423,300.556859,1.920965,1.006646,1.000456
...,...,...,...,...,...,...,...,...
39995,1.099,1.095,6.454068,541.036314,300.693744,2.695663,1.003315,1.000912
39996,1.099,1.096,6.438526,541.045591,300.693059,2.689171,1.003332,1.000910
39997,1.099,1.097,6.422989,541.054867,300.692375,2.682682,1.003349,1.000907
39998,1.099,1.098,6.407498,541.064146,300.691690,2.676212,1.003366,1.000905


In [6]:
df.to_csv('dataset.csv')